# Hands-on Tutorial V

## Introduction to thermal properties with GreenALM


In [ ]:
# add utils libary
import sys
from pathlib import Path

file = Path().cwd()
if file not in sys.path:
    sys.path.append(file)

from typing import Dict, Sequence
from pprint import pprint, pformat
from itertools import chain
    
# Load with magic commands
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
plt.rcParams["figure.figsize"] = (10,6)

import numpy as np

from pymatgen import Structure, IStructure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

from thermal_exp.core.eos import EosFactory
from thermal_exp.core.quasiharmonic import QuasiharmonicApprox

from gfalm.notebook.session import ParallelGreenAlmSession
from gfalm.notebook.session import init_client

from gfalm_utils.adapter import PymatgenStructureAdapter
from gfalm_utils.utils import change_concentrations

client = init_client('mpi')

print("Loading of modules was successful!")

### 1. Thermal expansion of Gold

In [ ]:
# Create pymatgen structure
structure = Structure(
    lattice=[[0.5, 0.5, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5]],
    species=['Au'],
    coords=[[0.0, 0.0, 0.0]],
)

In [ ]:
# Number of evaluation points
points = 8

# Set up the range for the calculations
volumes = np.linspace(14, 19, points)

# Temperature
temperature = 300



# Define an list for the free energies
free_energies = []

with ParallelGreenAlmSession(client, work_dir="workdir") as session:
    for idx, volume in enumerate(volumes):

        session.reset_pbar()
        session.write(f"Calculating for volume: {volume:.6f} ang^3", file=sys.stderr)

        base_input_params = session.read_input_file("input/base.in")
        result = session.run_main(
            PymatgenStructureAdapter(
                structure, base_input_params, volume=volume, name=f"au-{volume:.6f}"
            )
        )
        energy = result["general"].total_free_energy

        session.write(f"Free energy: {energy:.6f}", file=sys.stderr)

        free_energies.append(energy)
        
print("Calculations are ready!")

In [ ]:
eos_base = EosFactory("birch_murnaghan", volumes, free_energies)

debye_model = QuasiharmonicApprox(volumes, free_energies, structure)
thermal_energies = debye_model.vibrational_free_energies(temperature, volumes)
free_energies_total = thermal_energies + free_energies

eos = EosFactory("birch_murnaghan", volumes, free_energies_total)

def volume_to_lattice_constant(volume: float) -> float:
    return (volume * 4)**(1/3)

print(eos_base.volume_eq)
print(eos.volume_eq)
print(volume_to_lattice_constant(eos_base.volume_eq))
print(volume_to_lattice_constant(eos.volume_eq))
print(eos.bulk_modulus_eq.to("GPa"))

In [ ]:
### Plotting
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel(r"Volume [$\AA^3$]")
ax.set_ylabel("Energy [Ry]")
ax.grid(b=True, which="major", color="grey", alpha=0.2, linestyle="dashdot", lw=1.0)
ax.minorticks_on()
ax.grid(b=True, which="minor", color="lightgrey", alpha=0.3, ls="-", lw=0.5)
ax.plot(volumes, free_energies_total, "b+", label="$F(V)+F_{vib}(V)$", markersize=12)
ax.plot(volumes, free_energies, "rx", label="$F(V)$", markersize=12)
ax.axvline(eos_base.volume_eq, ls="--", color="red")
ax.axvline(eos.volume_eq, ls="--", color="blue")
ax.legend()
fig.tight_layout()
###

In [ ]:

# Set up tempertaures to evaluate 
temperatures = np.geomspace(1, 1000, num=50, endpoint=True)

equi_volumes = []

for temperature in temperatures:
    thermal_energies = debye_model.vibrational_free_energies(temperature, volumes)
    free_energies_total = thermal_energies + free_energies
    eos = EosFactory("birch_murnaghan", volumes, free_energies_total)
    equi_volumes.append(eos.volume_eq)

equi_volumes = np.array(equi_volumes)

alpha = np.gradient(equi_volumes, temperatures) / (equi_volumes)

fig, axes = plt.subplots(ncols=2)
fig.set_size_inches(14.5, 5)

axes[0].plot(temperatures, alpha, color="blue")
axes[1].plot(temperatures, equi_volumes, color="red")

axes[0].set_ylabel(r"$\alpha$")
axes[1].set_ylabel(r"Volume [$\AA^3$]")


for ax in axes:
    ax.set_xlabel('Temperature [K]')
    ax.grid(b=True, which="major", color="grey", alpha=0.2, linestyle="dashdot", lw=1.0)
    ax.minorticks_on()
    ax.grid(b=True, which="minor", color="lightgrey", alpha=0.3, ls="-", lw=0.5)
    
fig.tight_layout()



### 2. Ag-Au

#### 2.1. Equilbrium lattice constant

#### 2.2. Bulk modulus 

#### 2.3. Thermal expansion

In [ ]:

# Number of evaluation points
points = 8

# Set up the range for the calculations
volumes = np.linspace(14, 19, points)

# Temperature
temperature = 300

# Define method for calculating the equilibrum volume

def calculate_equilibrium_volume(
    structure: Structure, volumes: Sequence[float], base_input_params: Dict, session
) -> float:
    """
    Calculate the equilibriums volume for the given structure
    """
    energies = []
    base_name = ""

    for comp in structure.species_and_occu:
        base_name += "".join([f"{k}{v:.8f}" for k, v in comp.as_dict().items()])

    for vol in volumes:
        
        session.reset_pbar()
        session.write(f'{vol:6f}', file=sys.stderr)
        result = session.run_main(
            PymatgenStructureAdapter(
                structure,
                base_input_params,
                volume=vol,
                name=f"{base_name}-{vol:.6f}",
            )
        )
        energies.append(result["general"].total_free_energy)

    thermal_energies = QuasiharmonicApprox(
        volumes, energies, structure).vibrational_free_energies(temperature, volumes)

    
    eos = EosFactory("birch_murnaghan", volumes, thermal_energies + energies)
    volume = eos.volume_eq
    bulk_modulus = eos.bulk_modulus_eq.to("GPa")
    
    return {'volume': (volume * 4) ** (1/3),
            'bulk_modulus': bulk_modulus,
            'energies': energies}



In [ ]:
lattice=[[0.5, 0.5, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5]]
coords=[[0.0, 0.0, 0.0]]

gold_structure = IStructure(lattice, ['Au'], coords)
silver_structure = IStructure(lattice, ['Ag'], coords)

auag_structure_sets = [IStructure(lattice, [{'Au': c, 'Ag': 1 - c}], coords) for c in np.arange(0.2,0.8,0.2)]

results = {}



In [ ]:

with ParallelGreenAlmSession(client, work_dir="workdir") as session:
    base_input_params = session.read_input_file("input/base.in")

    for struct in [gold_structure, silver_structure]:
        
        results[struct] = calculate_equilibrium_volume(
            struct,
            volumes,
            base_input_params,
            session)
        
print("Done!!")


In [ ]:
print(results[gold_structure])

In [ ]:

with ParallelGreenAlmSession(client, work_dir="workdir") as session:
    base_input_params = session.read_input_file("input/base.in")

    for struct in auag_structure_sets:

        results[struct] = calculate_equilibrium_volume(
            struct,
            volumes,
            base_input_params,
            session)

print("Done!!")


In [ ]:


# Plot for the equilibriums volume
au_exp_a = 4.06
ag_exp_a = 4.08

fig, axes = plt.subplots(ncols=1)
fig.set_size_inches(14.5, 5)

axes.plot(1.0, au_exp_a, "ko", ms=10, zorder=8, clip_on=False)
axes.plot(0.0, ag_exp_a, "ko", ms=10, zorder=8, clip_on=False)

for struct in chain([gold_structure], auag_structure_sets, [silver_structure]):
    conc = struct[0].species.get('Au', 0.0)
    axes.plot(conc, results[struct]['volume'], "bo", ms=10, zorder=10, clip_on=False)

    
size= 12
axes.annotate('Au', (1.0, au_exp_a), (0.95, au_exp_a),fontsize=size)
axes.annotate('Ag', (0.0, ag_exp_a), (0.05, ag_exp_a), fontsize=size)

axes.set_xlabel('Au [atom.]')
axes.set_ylabel('Lattice constant [$\AA$]')
axes.grid(b=True, which="major", color="grey", alpha=0.2, linestyle="dashdot", lw=1.0)
axes.minorticks_on()
axes.grid(b=True, which="minor", color="lightgrey", alpha=0.3, ls="-", lw=0.5)
axes.set_xlim(0,1)

fig.tight_layout()


In [ ]:
# Plot bulk modulus

fig, axes = plt.subplots(ncols=1)
fig.set_size_inches(14.5, 5)

for struct in chain([gold_structure], auag_structure_sets, [silver_structure]):
    conc = struct[0].species.get('Au', 0.0)
    axes.plot(conc, results[struct]['bulk_modulus'], "bo", ms=10, zorder=10, clip_on=False)

axes.set_xlabel('Au [atom.]')
axes.set_ylabel('Bulkmodulus [GPa]')
axes.grid(b=True, which="major", color="grey", alpha=0.2, linestyle="dashdot", lw=1.0)
axes.minorticks_on()
axes.grid(b=True, which="minor", color="lightgrey", alpha=0.3, ls="-", lw=0.5)
axes.set_xlim(0,1)

fig.tight_layout()

In [ ]:
# Set up tempertaures to evaluate 
temperatures = np.geomspace(1, 1000, num=30, endpoint=True)

equi_volumes = []

fig, axes = plt.subplots(ncols=1)
fig.set_size_inches(14.5, 5)


structures = list(chain([gold_structure], auag_structure_sets, [silver_structure]))
colors = plt.get_cmap('viridis')(np.linspace(0,1,len(structures)))

for idx, struct in enumerate(structures):
        
    equi_volumes = []

    for temperature in temperatures:
        debye_model = QuasiharmonicApprox(volumes, results[struct]['energies'], struct)
        thermal_energies = debye_model.vibrational_free_energies(temperature, volumes)
        free_energies_total = thermal_energies + results[struct]['energies']
        eos = EosFactory("birch_murnaghan", volumes, free_energies_total)
        equi_volumes.append(eos.volume_eq)

    equi_volumes = np.array(equi_volumes)
    alpha = np.gradient(equi_volumes, temperatures) / (equi_volumes)

    axes.plot(temperatures, alpha, color=colors[idx], label={struct[0].species})

axes.set_ylabel(r"$\alpha$")
axes.set_ylabel(r"Volume [$\AA^3$]")

axes.set_xlabel('Temperature [K]')
axes.grid(b=True, which="major", color="grey", alpha=0.2, linestyle="dashdot", lw=1.0)
axes.minorticks_on()
axes.grid(b=True, which="minor", color="lightgrey", alpha=0.3, ls="-", lw=0.5)
axes.legend()
    
fig.tight_layout()

------------------------------------------------------------------------